### Galois Fields of degree 2

Burton Rosenberg

November 4, 2021



In [169]:

class QuadInt:
    
    def __init__(self,p,d):
        assert d%p!=0
        assert not self.qr(d,p)

        # p should be a prime but we only hope so
        assert p>2

        self.p = p
        self.d = d%p

    def qr(self,a,p):
        """
        check if a is quadradic residue mod p
        """
        return a**((p-1)//2)%p==1
        
    def get_mod(self):
        return self.p
    
    def get_des(self):
        return self.d

    def canonical(self,a):
        return (a[0]%self.p,a[1]%self.p)
    
    def zero_p(self,a):
        a = self.canonical(a)
        return a[0]==0 and a[1]==0
    
    def one_p(self,a):
        a = self.canonical(a)
        return a[0]==1 and a[1]==0
    
    def equal_p(self,a,b):
        a = self.canonical(a)
        b = self.canonical(b)
        return a[0]==b[0] and a[1]==b[1]

    def zero(self):
        return (0,0)
    
    def one(self):
        return (1,0)

    def conj(self,a):
        return (a[0],-a[1]%self.p)

    def minus(self,a):
        return (-a[0]%self.p,-a[1]%self.p)

    def add(self,a,b):
        c = (a[0]+b[0])%self.p
        d = (a[1]+b[1])%self.p
        return (c,d)
    
    def mult(self,a,b):
        c = (a[0]*b[0]+self.d*a[1]*b[1]) %self.p
        d = (a[0]*b[1]+a[1]*b[0]) %self.p
        return (c,d)
    
    def exp(self,a,n):
        if n==0:
            return (1,0)
        if n==1:
            return a
        if n%2==0:
            c = self.exp(a,n//2)
            return self.mult(c,c)
        c = self.exp(a,n-1)
        return self.mult(a,c)
 
    def extended_gcd(self,a,b):
        """
        extended GCD algorithm. recursive.
        returns (d,s,t) where d = s*a+t*b 
        and d = gcd(a,b)
        """
        assert(
            a>=0 and b>=0 )
        if b==0:
            return (a,1,0)
        (q,r) = divmod(a,b)
        (d,s,t) = self.extended_gcd(b,r)
        # gcd(a, b) == gcd(b, r) == s*b + t*r == s*b + t*(a - q*b)
        return (d,t,s-q*t)

    def invert_base(self,c):
        assert c[1]==0
        a = c[0]
        (d,t,s) = self.extended_gcd(a,self.p)
        assert 1==d
        return (t%self.p,0)
    
    def invert(self,a):
        if self.zero_p(a):
            return (0,0)

        c = (a[0],-a[1])
        d = self.mult(a,c)
        d_inv = self.invert_base(d)
        return self.mult(c,d_inv)
        
    def next(self,a):
        if a[1]==(self.p-1):
            return ((a[0]+1)%self.p,0)
        return (a[0],(a[1]+1)%self.p)
        
    def __str__(self):
        return f'Z/{self.p}Z[rad({self.d})]'


In [170]:
qi = QuadInt(11,3)
print(qi)

def test_inv(qi):
    p = qi.get_mod()
    a = qi.one()
    while not qi.zero_p(a):
        b = qi.invert(a)
        c = qi.mult(a,b)
        print(f'{a} inverse is {b}')
        assert qi.one_p(c), f'{a}'
        a = qi.next(a)

test_inv(qi)

Z/5Z[rad(3)]
(1, 0) inverse is (1, 0)
(1, 1) inverse is (2, 3)
(1, 2) inverse is (4, 2)
(1, 3) inverse is (4, 3)
(1, 4) inverse is (2, 2)
(2, 0) inverse is (3, 0)
(2, 1) inverse is (2, 4)
(2, 2) inverse is (1, 4)
(2, 3) inverse is (1, 1)
(2, 4) inverse is (2, 1)
(3, 0) inverse is (2, 0)
(3, 1) inverse is (3, 4)
(3, 2) inverse is (4, 4)
(3, 3) inverse is (4, 1)
(3, 4) inverse is (3, 1)
(4, 0) inverse is (4, 0)
(4, 1) inverse is (3, 3)
(4, 2) inverse is (1, 2)
(4, 3) inverse is (1, 3)
(4, 4) inverse is (3, 2)


In [129]:
for i in range(25):
    print(qi.exp((0,2),i))

(1, 0)
(0, 2)
(2, 0)
(0, 4)
(4, 0)
(0, 3)
(3, 0)
(0, 1)
(1, 0)
(0, 2)
(2, 0)
(0, 4)
(4, 0)
(0, 3)
(3, 0)
(0, 1)
(1, 0)
(0, 2)
(2, 0)
(0, 4)
(4, 0)
(0, 3)
(3, 0)
(0, 1)
(1, 0)


In [130]:
p = qi.get_mod()
a = qi.one()
while not qi.zero_p(a):
    b1 = qi.mult(a,a)
    a_minus = qi.minus(a)
    b2 = qi.mult(a_minus,a_minus)
    assert qi.equal_p(b1,b2)
    print(f'{a} or {a_minus} squared is {b1}')
    a = qi.next(a)
        

(1, 0) or (4, 0) squared is (1, 0)
(1, 1) or (4, 4) squared is (4, 2)
(1, 2) or (4, 3) squared is (3, 4)
(1, 3) or (4, 2) squared is (3, 1)
(1, 4) or (4, 1) squared is (4, 3)
(2, 0) or (3, 0) squared is (4, 0)
(2, 1) or (3, 4) squared is (2, 4)
(2, 2) or (3, 3) squared is (1, 3)
(2, 3) or (3, 2) squared is (1, 2)
(2, 4) or (3, 1) squared is (2, 1)
(3, 0) or (2, 0) squared is (4, 0)
(3, 1) or (2, 4) squared is (2, 1)
(3, 2) or (2, 3) squared is (1, 2)
(3, 3) or (2, 2) squared is (1, 3)
(3, 4) or (2, 1) squared is (2, 4)
(4, 0) or (1, 0) squared is (1, 0)
(4, 1) or (1, 4) squared is (4, 3)
(4, 2) or (1, 3) squared is (3, 1)
(4, 3) or (1, 2) squared is (3, 4)
(4, 4) or (1, 1) squared is (4, 2)
